In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from time import sleep
from bs4 import BeautifulSoup
import csv

print('-Finish importing package')

In [ ]:
#Step 1: Login to Linkedin

#Open Chrome and login Linked login site
driver = webdriver.Chrome()
driver.maximize_window()
url = 'https://www.linkedin.com/login'
driver.get(url)
print('-Fishish initializing a driver')
sleep(2)

In [ ]:
#Import username and password
credential = open('login_credential.txt')
line = credential.readlines()
username = line[0]
password = line[1]
print('-Finish importing the login credential')

#Key in username
email_field = driver.find_element(By.ID, 'username')
email_field.send_keys(username)
print('-Finish keying in email')
sleep(3)

#Key in password
password_field = driver.find_element(By.NAME, 'session_password')
password_field.send_keys(password)
print('-Finish keying in password')
sleep(2)

#Click login button
login_field = driver.find_element(By.XPATH, '//*[@id="organic-div"]/form/div[3]/button')
#Lấy xPath = chọn trỏ phải phần tử html-> chọn copy-> copy xPath
login_field.click()
print('-Finish logging in')
sleep(3)

In [ ]:
#Step 2: Search for the profile we want to crawl

#Locate the search bar element
search_field = driver.find_element(By.CLASS_NAME,'search-global-typeahead__input')
#Input the search query to the search bar
search_query = input('What profile do you want to scrape?')
search_field.send_keys(search_query)

#Search
search_field.send_keys(Keys.RETURN)
sleep(5)
print('-Finish searching ...')

tab_people = driver.find_element(By.XPATH, '//*[@id="search-reusables__filters-bar"]/ul/li[2]/button')
tab_people.send_keys(Keys.RETURN)

In [50]:
# Task 3: Scrape the URLs of the profiles

# Task 3.1: Write a function to extract the URLs of one page
def GetURL():
    page_source = BeautifulSoup(driver.page_source)
    profiles = page_source.find_all('a', class_ = 'search-result__result-link ember-view')
    all_profile_URL = []
    for profile in profiles:
        profile_ID = profile.get('href')
        profile_URL = "https://www.linkedin.com" + profile_ID
        if profile_URL not in all_profile_URL:
            all_profile_URL.append(profile_URL)
    return all_profile_URL

# Task 3.2: Navigate through many page, and extract the profile URLs of each page

input_page = int(input('How many pages you want to scrape: '))
URLs_all_page = []
index = 1
for page in range(input_page):
    URLs_one_page = GetURL()
    sleep(2)
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight);') #scroll to the end of the page
    sleep(3)
    #next_button = driver.find_element(By.CLASS_NAME,"artdeco-pagination__button--next")
    index+=1
    next_button = driver.find_element(By.XPATH,f'//button[@aria-label="Page {index}"]')
    driver.execute_script("arguments[0].click();", next_button)
    URLs_all_page = URLs_all_page + URLs_one_page
    sleep(2)
print(URLs_all_page)
print('- Finish Task 3: Scrape the URLs')


[]
- Finish Task 3: Scrape the URLs


In [52]:
# Task 4: Scrape the data of 1 Linkedin profile, and write the data to a .CSV file
with open('output.csv', 'w',  newline = '') as file_output:
    headers = ['Name', 'Job Title', 'Location', 'URL']
    writer = csv.DictWriter(file_output, delimiter=',', lineterminator='\n',fieldnames=headers)
    writer.writeheader()
    for linkedin_URL in URLs_all_page:
        driver.get(linkedin_URL)
        print('- Accessing profile: ', linkedin_URL)
        sleep(3)
        page_source = BeautifulSoup(driver.page_source, "html.parser")
        info_div = page_source.find('div',{'class':'flex-1 mr5'})
        try:
            name = info_div.find('li', class_='inline t-24 t-black t-normal break-words').get_text().strip() #Remove unnecessary characters 
            print('--- Profile name is: ', name)
            location = info_div.find('li', class_='t-16 t-black t-normal inline-block').get_text().strip() #Remove unnecessary characters 
            print('--- Profile location is: ', location)
            title = info_div.find('h2', class_='mt1 t-18 t-black t-normal break-words').get_text().strip()
            print('--- Profile title is: ', title)
            writer.writerow({headers[0]:name, headers[1]:location, headers[2]:title, headers[3]:linkedin_URL})
            print('\n')
        except:
            pass

print('Mission Completed!')

Mission Completed!
